# PoSyg Consensus Protocol Analysis

Author: Daniil Krizhanovskyi  
Date: October 2024  
Description: Jupyter/Python notebook for in-depth analysis of Proof-of-Synergy (PoSyg) agent-based simulation results: network health, validator strategies, synergy scoring, attack resistance, economics, and statistical tests.

## Notebook Structure Manifest
This notebook documents the analytical pipeline used in the PoSyg consensus analysis.
---

### 1. Load Simulation Data
- Import metrics, validator states, and configuration JSON for each scenario.
- Supported scenarios: baseline, cartel_attack, high_byzantine.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import json
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [ ]:
def load_simulation_data(scenario_name):
    """Load simulation results for a given scenario"""
    base_path = f"results_{scenario_name}"
    
    # Load metrics history
    metrics_df = pd.read_csv(f"{base_path}/metrics_history.csv")
    
    # Load final validator states
    validators_df = pd.read_csv(f"{base_path}/final_validator_states.csv")
    
    # Load configuration
    with open(f"{base_path}/simulation_config.json", "r") as f:
        config = json.load(f)
    
    return metrics_df, validators_df, config

# Load all scenarios
scenarios = ["baseline", "cartel_attack", "high_byzantine"]
results = {}

for scenario in scenarios:
    try:
        metrics, validators, config = load_simulation_data(scenario)
        results[scenario] = {
            "metrics": metrics,
            "validators": validators,
            "config": config
        }
        print(f"Loaded {scenario}: {len(metrics)} epochs, {len(validators)} validators")
    except FileNotFoundError:
        print(f"Warning: Results for {scenario} not found")

### 2. Network Health Analysis
- Plot active validator count per epoch.
- Plot finality rate (10-epoch moving average).
- Plot Gini coefficient for stake distribution.
- Plot cumulative slashing events.
- Compare metrics across all scenarios.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle("Network Health Metrics Across Scenarios", fontsize=16)

# Plot 1: Active validators over time
ax = axes[0, 0]
for scenario, data in results.items():
    metrics = data["metrics"]
    ax.plot(metrics["epoch"], metrics["active_validators"], 
            label=scenario, linewidth=2)
ax.set_xlabel("Epoch")
ax.set_ylabel("Active Validators")
ax.set_title("Active Validator Count")
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Finality rate
ax = axes[0, 1]
for scenario, data in results.items():
    metrics = data["metrics"]
    # Calculate moving average
    window = 10
    finality_ma = metrics["finality_rate"].rolling(window=window, min_periods=1).mean()
    ax.plot(metrics["epoch"], finality_ma, 
            label=f"{scenario} (MA-{window})", linewidth=2)
ax.set_xlabel("Epoch")
ax.set_ylabel("Finality Rate")
ax.set_title("Block Finality Rate (Moving Average)")
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.1)

# Plot 3: Gini coefficient (decentralization)
ax = axes[1, 0]
for scenario, data in results.items():
    metrics = data["metrics"]
    ax.plot(metrics["epoch"], metrics["gini_coefficient"], 
            label=scenario, linewidth=2)
ax.set_xlabel("Epoch")
ax.set_ylabel("Gini Coefficient")
ax.set_title("Stake Distribution Inequality")
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1)

# Plot 4: Slashing events
ax = axes[1, 1]
for scenario, data in results.items():
    metrics = data["metrics"]
    ax.plot(metrics["epoch"], metrics["slashing_events"], 
            label=scenario, linewidth=2)
ax.set_xlabel("Epoch")
ax.set_ylabel("Cumulative Slashing Events")
ax.set_title("Protocol Violations (Slashing)")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3. Validator Strategy Analysis
- Group validators by strategy (honest, byzantine, cartel, etc.).
- Aggregate and plot average stake, synergy score, blocks proposed, and slashing per strategy.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle("Validator Performance by Strategy", fontsize=16)

for idx, (scenario, data) in enumerate(results.items()):
    ax = axes[idx]
    validators = data["validators"]
    
